In [1]:
from init import *
import time

In [2]:
model = Model()


In [3]:
m = mcsdk.examples.BuildModel_Simple()
m.evaluations.append(EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentBrownianMotion"}, {"name": "SimpleBrownianMotion", "start": {"name": "start", "value": 0.1}, "args": [{"name": "drift", "value": 0.2}, {"name": "diffusion", "value": 2}]}], "evaluations": [{"state": 0, "time": 2}]}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'mean': [0.5230805277824402],
 'names': ['SimpleBrownianMotion'],
 'npaths': [10000],
 'skewness': [-0.015936588868498802],
 'stddev': [2.853414535522461],
 'time_points': [2.0],
 'time_steps': [10]}

In [5]:
m = mcsdk.examples.BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 10000
for t in [1,2,5,10]:
    m.evaluations.append(EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'mean': [0.29977932572364807,
  0.5001074075698853,
  1.0960489511489868,
  2.0979936122894287],
 'names': ['SimpleBrownianMotion'],
 'npaths': [1000000, 1000000, 1000000, 1000000],
 'skewness': [0.00254398793913424,
  -0.0002535203821025789,
  -3.73982620658353e-05,
  -0.002090287860482931],
 'stddev': [2.0001721382141113,
  2.826321840286255,
  4.471787452697754,
  6.330761432647705],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [1000, 2000, 5000, 10000]}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

3.1 nanoseconds to make a single time step


In [7]:
m = mcsdk.examples.BadModel()
m.NumPaths = 1000000
m.TimeSteps = 10000
for t in [1,2,5,10]:
    m.evaluations.append(EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'error': ['../src/binding/json/all.cpp void from_json(const json&, Model&) AddEvaluationPoint']}

# Two processes, Three evaluations

In [8]:
model = mcsdk.examples.TwoProcessesModel()
model.TimeSteps = 10000
for time in [1,2,5]:
    model.evaluations.append(EvaluationPoint(state=0,time=time))
r = requests.post(f'{server}/model',model.json())
r.json()

{'mean': [0.3017081916332245,
  0.2891724705696106,
  0.5055649280548096,
  0.480943500995636,
  1.0719245672225952,
  1.1176369190216064],
 'names': ['SimpleBrownianMotion', 'SimpleBrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'skewness': [-0.008047826588153839,
  0.0152851277962327,
  -0.0011226338101550937,
  0.019466329365968704,
  0.04990454763174057,
  0.03486330434679985],
 'stddev': [1.9988871812820435,
  1.9962352514266968,
  2.8082852363586426,
  2.7867581844329834,
  4.504989147186279,
  4.473876476287842],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [9]:
er = EvaluationResults(r.json(),model)
er

2 states with 3 evaluations

In [10]:
er.df()

,name,state,time,step,npaths,mean,mean_error,stddev,skewness
0,SimpleBrownianMotion,0,1.0,2000,10000,0.301708,0.019989,1.998887,-0.008048
1,SimpleBrownianMotion,1,1.0,2000,10000,0.289172,0.019962,1.996235,0.015285
2,SimpleBrownianMotion,0,2.0,4000,10000,0.505565,0.028083,2.808285,-0.001123
3,SimpleBrownianMotion,1,2.0,4000,10000,0.480944,0.027868,2.786758,0.019466
4,SimpleBrownianMotion,0,5.0,10000,10000,1.071925,0.045050,4.504989,0.049905
5,SimpleBrownianMotion,1,5.0,10000,10000,1.117637,0.044739,4.473876,0.034863
